In [7]:
import pickle
from collections import Counter
from multiprocessing.pool import Pool

import numpy as np
import pandas as pd

from countries import get_countries

In [ ]:
with open("data/actors.pkl", "rb") as f:
    df = pd.DataFrame(sorted(pickle.load(f)), columns=["nconst"])

def parallelize_dataframe(df, func, n_cores=12):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [2]:
df = parallelize_dataframe(df, get_countries)

In [3]:
df.to_pickle("data/birth_countries.pkl")

,nconst,birth_country
0,nm0000001,USA
1,nm0000002,USA
2,nm0000003,France
3,nm0000006,Sweden
4,nm0000007,USA
...,...,...
13414,nm9656000,
13415,nm9706424,
13416,nm9712684,
13417,nm9845146,India


In [14]:
df_ = pd.read_pickle("data/birth_countries.pkl")
countries = Counter(df_["birth_country"].values).most_common()
for i, country in enumerate(countries):
    print(i, country)

0 ('USA', 2710)
1 ('', 2638)
2 ('Japan', 968)
3 ('UK', 635)
4 ('France', 585)
5 ('India', 583)
6 ('Italy', 497)
7 ('Germany', 394)
8 ('Turkey', 330)
9 ('Mexico', 279)
10 ('China', 272)
11 ('British India', 261)
12 ('Spain', 259)
13 ('Russia', 207)
14 ('Philippines', 206)
15 ('Canada', 157)
16 ('Egypt', 147)
17 ('Sweden', 126)
18 ('Argentina', 122)
19 ('Austria', 116)
20 ('Greece', 114)
21 ('Czech Republic', 109)
22 ('Hong Kong', 104)
23 ('Poland', 102)
24 ('Denmark', 100)
25 ('Taiwan', 89)
26 ('Australia', 75)
27 ('Hungary', 75)
28 ('Brazil', 70)
29 ('Iran', 61)
30 ('South Korea', 57)
31 ('Romania', 56)
32 ('Ukraine', 52)
33 ('Yugoslavia', 47)
34 ('Indonesia', 41)
35 ('Ireland', 39)
36 ('Belgium', 38)
37 ('Switzerland', 32)
38 ('Pakistan', 30)
39 ('Netherlands', 27)
40 ('Finland', 27)
41 ('Bulgaria', 23)
42 ('Austria-Hungary', 22)
43 ('British Crown Colony', 22)
44 ('Cuba', 21)
45 ('Korea', 18)
46 ('Algeria', 17)
47 ('Norway', 17)
48 ('Slovakia', 16)
49 ('Malaysia', 16)
50 ('Bangladesh